In [1]:
import torch

# Test Area

以向量第二項的計算為例

In [2]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89],
     [0.55, 0.87, 0.66],
     [0.57, 0.85, 0.64],
     [0.22, 0.58, 0.33],
     [0.77, 0.25, 0.10],
     [0.05, 0.80, 0.55]]
)

query =  inputs[1] # 查詢目標
attention_score = torch.empty(inputs.shape[0]) # 建立空的注意力分數張量

In [3]:
for i, i_value in enumerate(inputs): # 迭代x_i
    attention_score[i] = torch.dot(i_value, query) # 計算查詢目標與每個鍵的點積
    
print("Attention Score: ", attention_score)

Attention Score:  tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [ ]:
# Simple ormalization
attention_weight = attention_score / attention_score.sum() # 計算注意力權重(總和為1)

print("Attention Weight: ", attention_weight)
print("Sum of Attention Weight: ", attention_weight.sum())


Attention Weight:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum of Attention Weight:  tensor(1.0000)


In [5]:
# Softmax Normalization
def softmax_native(x):
    """
        softmax函式進行正規化
    """
    return torch.exp(x) / torch.exp(x).sum(dim= 0)

attention_weight_softmax_naive = softmax_native(attention_score)

print("Softmax Attention Weight: ", attention_weight_softmax_naive)
print("Sum of Softmax Attention Weight: ", attention_weight_softmax_naive.sum())

Softmax Attention Weight:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum of Softmax Attention Weight:  tensor(1.)


In [6]:
# PyTorch Softmax Normalization
attention_weight_softmax = torch.softmax(attention_score, dim= 0)

print("Softmax Attention Weight (PyTorch): ", attention_weight_softmax)
print("Sum of Softmax Attention Weight (PyTorch): ", attention_weight_softmax.sum())

Softmax Attention Weight (PyTorch):  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum of Softmax Attention Weight (PyTorch):  tensor(1.)


In [ ]:
context_vec = torch.zeros(query.shape)

for i, i_value in enumerate(inputs):
    context_vec += attention_weight_softmax[i] * i_value # 計算加權後的上下文向量
print("Context Vector: ", context_vec)

Context Vector:  tensor([0.4419, 0.6515, 0.5683])


# Regular Code

In [13]:
# Caculate Context vector of all inputs
attention_scores = torch.empty((inputs.shape[0], inputs.shape[0]))

"""
for i, i_value in enumerate(inputs):
    for j, j_value in enumerate(inputs):
        attention_scores[i,j] = torch.dot(i_value, j_value)
"""
attention_scores = inputs @ inputs.T # 使用矩陣乘法計算注意力分數(較for迴圈迅速)

print(attention_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [ ]:
attention_weights = torch.softmax(attention_scores, dim=-1) # dim函數指定處理的維度，dim=-1表示要求最後一個維度的softmax

print(attention_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [14]:
all_context_vec = attention_weights @ inputs # 使用矩陣乘法計算所有上下文向量

print(all_context_vec)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [ ]:
# Todo: 模組化